In [175]:
import sqlite3
import pandas as pd 
import requests

con =sqlite3.connect('database.sqlite')
cursor = con.cursor()

cursor.execute(
"""
SELECT name
FROM sqlite_master
WHERE type='table';
"""
)
print(cursor.fetchall())

[('sqlite_sequence',), ('Matches',), ('Teams_in_Matches',), ('Teams',), ('Unique_Teams',)]


In [176]:
df_new = pd.read_sql_query("""
    SELECT  DISTINCT *,
    CASE WHEN m.FTHG > FTAG THEN 1
                     ELSE 0
                     END AS Win,
    CASE WHEN m.Div = 'E0' THEN 'English Premier League' ELSE 'Bundesliga' END as League
    FROM Matches AS m
    WHERE m.season = '2012'
    """, con)
df_new['New_Date'] = df_new['Date']+'T00:00:00'
#dates =df_new['New_Date'].unique()
df_new.sample(20)


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Win,League,New_Date
893,45156,E0,2012,2013-03-09,QPR,Sunderland,3,1,H,1,English Premier League,2013-03-09T00:00:00
557,2261,D2,2012,2012-08-11,St Pauli,Ingolstadt,1,1,D,0,Bundesliga,2012-08-11T00:00:00
789,45052,E0,2012,2012-12-23,Chelsea,Aston Villa,8,0,H,1,English Premier League,2012-12-23T00:00:00
959,45222,E0,2012,2013-05-04,Fulham,Reading,2,4,A,0,English Premier League,2013-05-04T00:00:00
328,2032,D2,2012,2012-10-21,Paderborn,St Pauli,1,1,D,0,Bundesliga,2012-10-21T00:00:00
518,2222,D1,2012,2012-11-28,Nurnberg,Hoffenheim,4,2,H,1,Bundesliga,2012-11-28T00:00:00
484,2188,D2,2012,2012-11-02,Regensburg,Paderborn,0,2,A,0,Bundesliga,2012-11-02T00:00:00
719,44982,E0,2012,2012-11-11,Man City,Tottenham,2,1,H,1,English Premier League,2012-11-11T00:00:00
533,2237,D2,2012,2012-08-26,Ingolstadt,Frankfurt FSV,0,2,A,0,Bundesliga,2012-08-26T00:00:00
381,2085,D2,2012,2013-05-06,Braunschweig,Cottbus,0,0,D,0,Bundesliga,2013-05-06T00:00:00


### 

In [186]:
def pipeline(season):
    df_new = pd.read_sql_query(f"""SELECT  DISTINCT m.Match_ID, m.Season, 
    m.Date, m.HomeTeam, m.AwayTeam, m.FTHG, m.FTAG, m.Div,
    CASE WHEN m.FTHG > FTAG THEN 1 ELSE 0 END AS WinHome,
    CASE WHEN m.FTHG < FTAG THEN 1 ELSE 0 END AS WinAway,
    CASE WHEN m.Div == 'E0' THEN 'English Premier League' ELSE 'Bundesliga' END as League
    FROM Matches AS m
    JOIN Teams_IN_matches AS tm ON M.Match_ID = tm.Match_ID
    JOIN Unique_Teams AS ut ON tm.Unique_Team_ID = ut.Unique_Team_ID
    JOIN Teams as t on ut.TeamName = t.TeamName
    WHERE m.season = {season}
    ORDER BY m.Match_ID 
    LIMIT 3
    """, con)
    df_new['New_Date'] = df_new['Date']+'T00:00:00'
    
    urls=[]   
    for date in df_new['New_Date']:
        urls.append(f'https://api.darksky.net/forecast/{password}/52.52,13.405,{date}?exclude=minutely,hourly, alerts, flags')   
        pt_dict =[]
        for url in urls:
            response = requests.get(url)
            if response.status_code == 200:
                response_dict = response.json()
                pt_dict.append(response_dict['daily']['data'][0]['precipIntensity'])
    df_new['Rain']= pt_dict
    df_new['Rain'][df_new['Rain'] >0]= 1
    df_new['Rain'][df_new['Rain'] == 0]= 0
    df_home =df_new[['HomeTeam', 'League', 'Season', 'FTHG','WinHome', 'Rain']]
    df_home.rename(columns={'HomeTeam':'TeamName', 'FTHG': 'Goal', 'WinHome': 'Win'}, inplace =True)
    df_away =df_new[['AwayTeam', 'League', 'Season', 'FTAG','WinAway','Rain']]
    df_away.rename(columns={'AwayTeam':'TeamName', 'FTAG': 'Goal', 'WinAway': 'Win'}, inplace=True)
    df_com = pd.concat([df_home, df_away], ignore_index=False)
    df_com.loc[(df_com['Win'] ==1)& (df_com['Rain'] ==1), 'WinRainday'] =1 

    df_agg =df_com.groupby(['TeamName', 'League', 'Season']).agg({
       'Goal':'sum', 'Win':'sum', 'Date':'count', 'WinRainday': 'sum'})
    df_agg['WinPercentRainDay']=(df_agg['WinRainday']/df_agg.iloc[:,-2])
    df_final =df_new1
    return df_final

Data = pipeline(2009)
Data

ValueError: Length of values does not match length of index